**<span style="color:red">IMPORTANT:</span>** This notebook requires [`BSpline2.scala`](http://cobweb.cs.uga.edu/~mec/BSpline2.scala) to be placed in the root of your JupyterHub workspace. If you're on `hub.aods.io`, the you can do this using the following AODS Upload link: http://upload.aods.io/http://cobweb.cs.uga.edu/~mec/BSpline2.scala.

# Regression Splines
Suppose you want to approximate the function $x(t)$ for some process that you have sampled over the input domain (usually time). If the process isn't linear, then directly using multiple linear regression may not be a good idea. Instead, you can better model the curvature of the process using a regression spline.

With a regression spline, the function is modeled as a linear combination of basis functions:

$$ f(t) = \sum_i c_i \phi_i(t) . $$

While many different types of basis functions exist, it is common to use [B-spline](https://en.wikipedia.org/wiki/B-spline#Definition) basis functions. The remainder of this notebook demonstrates how to reformulate a regression spline model as a multiple linear regression in order to approximate a function for the number (in thousands) of Australian residents measured quarterly from March 1971 to March 1994. To do this, we are going use the [Quarterly Time Series of the Number of Australian Residents](https://stat.ethz.ch/R-manual/R-devel/library/datasets/html/austres.html) dataset from the R [`datasets`](https://stat.ethz.ch/R-manual/R-devel/library/datasets/html/00Index.html) package. For convenience, a copy of this dataset is provided at http://cobweb.cs.uga.edu/~mec/austres.csv.

First, let's import some packages from ScalaTion:

In [ ]:
import scalation.analytics._     // for regression
import scalation.analytics.fda._ // for functional data analysis
import scalation.linalgebra._    // for linear algebra support
import scalation.relalgebra._    // for relational algebra support
:load BSpline2.scala

Next, let's load in the dataset using a [`Relation`](http://cobweb.cs.uga.edu/~jam/scalation_1.3/scalation_mathstat/target/scala-2.12/api/scalation/relalgebra/Relation$.html) to see what's available:

In [ ]:
val url = "http://cobweb.cs.uga.edu/~mec/austres.csv" // dataset url
val rel = Relation(url, "austres", "SDD", 0, ",")     // create relation
rel.show()

It looks like we want to use the second column (index 1) to represent points in the input domain and the last column (index 2) to represent our response. Let's extract them as vectors:

In [ ]:
val t = rel.toVectorD(1) // input / time point vector
val y = rel.toVectorD(2) // response vector

Now, let's prepare for the creation of our B-spline basis functions. The degree of a B-spline basis function is one less than its order. Therefore, we'll use $k=4$ to denote the order for a cubic B-spline. Furthermore B-spline basis functions are parameterized according to a vector of non-decreasing "knots" across the input domain. The regression spline will only be defined on the inclusive interval defined by the first and last elements of the knot vector. As described in [Patrikalakis, N. M., & Maekawa, T. (2009)](http://web.mit.edu/hyperbook/Patrikalakis-Maekawa-Cho/node17.html), it is often desirable to "clamp" the knot vector by repeating the first and last time points $k$ times to ensure that the resulting regression spline passes through the first and last points in the response. **<span style="color:blue">NOTE:</span>** We encourage the user to try the remaining steps in this notebook with different sets of internal knots (keeping the endpoints the same).

In [ ]:
val k     = 4                        // order 4; degree 3
val knots = BSpline2Util.clamp(k, t) // clamped knot vector
val bs    = new BSpline2(knots, k)   // cubic b-spline basis generator

Now we are ready to define $\phi_i(t)$:

In [ ]:
def phi (i: Int)(t: Double) = bs(k)(i)(t)

To reformulate the regression spline model as a multiple linear regression model, we need to construct a design matrix using the basis functions. Each column in the matrix will represent one of the basis functions. Each row will represent each of the basis functions evaluated at a particular input point in the closed interval defined by the knot vector. Then, we have:

$$ c(t) = Xc, $$

which resembles, more closely, multiple linear regression. Let's create the design matrix and solve for the coefficient vector using the usual OLS method:

In [ ]:
val X = new MatrixD (t.dim, bs.count())                   // create empty design matrix
for (j <- X.range1; i <- X.range2) X(j, i) = phi(i)(t(j)) // evaluate each spline at each input point
println(X)                                                // show the design matrix
val c = (x.t * x).inverse * x.t * y                       // solve for x using OLS

Before we define $f(t)$ using the regression spline, let's look at the residual of the multiple linear regression model:

In [ ]:
val z   = X * c   // predicted response
val e   = y - z   // residual
val sse = e dot e // sum of squared error

Now, let's define $f(t)$ using the regression spline:

In [ ]:
def f(t: Double) = (0 until bs.count()).map(i => c(i) * phi(i)(t)).sum 

Now you should be able to compute values anywhere inside the interval.

## References
* [1] Patrikalakis, N. M., & Maekawa, T. (2009). *Shape Interrogation for Computer Aided Design and Manufacturing.* Springer Science & Business Media.
* [2] P. J. Brockwell and R. A. Davis (1996). *Introduction to Time Series and Forecasting.* Springer